# Histopathologic Cancer Detection
###### Identify metastic tissue in histopathologic scans of lymph node sections

## Imports

In [3]:
%matplotlib inline

import os
import cv2
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.nasnet import NASNetLarge, NASNetMobile
from tensorflow.keras.layers import Dense, Activation, Input, Lambda
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint

## Parameters

In [ ]:
model_name = "ResNet50_pretrained"

## Load Data

In [ ]:
train_X = []
train_y = []

df = pd.read_csv('dataset\\train_labels.csv')

for index, row in df.iterrows():
    img_id = row["id"]
    train_X.append(cv2.imread(f'dataset\\train\\{img_id}.tif'))
    
    train_y.append(row["label"])
    
train_X = np.array(train_X)
train_y = np.array(train_y)
    

In [ ]:
print(train_X.shape)
print(train_y.shape)

## Display a randomly selected image

In [ ]:
for img in train_X[np.random.choice(len(train_X), size=1, replace=False)]:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

## Define Models

### ResNet50 - Untrained

In [ ]:
img_input = Input(shape=(96,96,3))
norm = Lambda(lambda x: ((x/255.0) - 0.5))(img_input)

resnet = ResNet50(include_top=False, weights=None,input_tensor=norm, input_shape=(96,96,3), pooling='max')
x = resnet.output
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs=img_input, outputs=output)
print(model.summary())

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
checkpoint = ModelCheckpoint("model.h5", monitor='val_acc', verbose=1,save_best_only=False, mode='max')

### ResNet50 - Pretrained 

In [ ]:
img_input = Input(shape=(96,96,3))
norm = Lambda(lambda x: ((x/255.0) - 0.5))(img_input)

resnet = ResNet50(include_top=False, weights='imagenet',input_tensor=norm, input_shape=(96,96,3), pooling='max')
x = resnet.output
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs=img_input, outputs=output)
print(model.summary())

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
checkpoint = ModelCheckpoint("models\\" + model_name + "_{epoch:03d}_{val_acc:.4f}.h5", monitor='val_acc', verbose=1,save_best_only=False, mode='max')

### InceptionV3 - Pretrained

In [ ]:
model_name = "InceptionV3_pretrained"
img_input = Input(shape=(96,96,3))
norm = Lambda(lambda x: ((x/255.0) - 0.5))(img_input)
 
inception = InceptionV3(include_top=False, weights='imagenet',input_tensor=norm, input_shape=(96,96,3), pooling='max')
x = inception.output
x = Dense(1)(x)
output = Activation('sigmoid')(x)
 
model = Model(inputs=img_input, outputs=output)
print(model.summary())
 
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
checkpoint = ModelCheckpoint("models\\" + model_name + "_{epoch:03d}_{val_acc:.4f}.h5", monitor='val_acc', verbose=1,save_best_only=False, mode='max')

### NASNetLarge - Pretrained

In [ ]:
model_name = "NASNetLarge_pretrained"
img_input = Input(shape=(96,96,3))
norm = Lambda(lambda x: ((x/255.0) - 0.5))(img_input)
 
nasnet_large = NASNetLarge(include_top=False, weights='imagenet',input_tensor=norm, input_shape=(96,96,3), pooling='max')
x = nasnet_large.output
x = Dense(1)(x)
output = Activation('sigmoid')(x)
 
model = Model(inputs=img_input, outputs=output)
print(model.summary())
 
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
checkpoint = ModelCheckpoint("models\\" + model_name + "_{epoch:03d}_{val_acc:.4f}.h5", monitor='val_acc', verbose=1,save_best_only=False, save_weights_only=True, mode='max')

### NASNetMobile - Pretrained

In [ ]:
model_name = "NASNetMobile_pretrained"
 
img_input = Input(shape=(96,96,3))
norm = Lambda(lambda x: ((x/255.0) - 0.5))(img_input)
 
nasnet = NASNetMobile(include_top=False, weights='imagenet',input_tensor=norm, input_shape=(96,96,3), pooling='max')
x = nasnet.output
x = Dense(1)(x)
output = Activation('sigmoid')(x)
 
model = Model(inputs=img_input, outputs=output)
print(model.summary())
 
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
checkpoint = ModelCheckpoint("models\\" + model_name + "_{epoch:03d}_{val_acc:.4f}.h5", monitor='val_acc', verbose=1,save_best_only=False, mode='max')

## Train Model 

In [ ]:
history = model.fit(train_X, train_y, validation_split=0.2, callbacks=[checkpoint], epochs=50)

## Plot and Save Results

In [ ]:
fig_acc = plt.figure()
ax1 = plt.subplot(111)
ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
plt.title('Training vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.savefig("figures\\" + model_name + "_acc.png")
plt.show()


fig_loss = plt.figure()
ax2 = plt.subplot(111)
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
plt.title('Training vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig("figures\\" + model_name + "_loss.png")
plt.show()

ResNet50_pretrained